# The Python `set` type, and a bit about `dict`s as well

* Share $\mathcal{O}(1)$ traits with `dict`s
* Useful features once you get to know them
* Underappreciated

![](./python_sets.png)

## Properties

* Membership check is $\mathcal{O}(1)$ on average ($\mathcal{O}(n)$ is worst case)
    - `x in my_set`

* Unordered collection, making it useful when comparing collections for just _membership_
    - `{'a', 'b', 'c'} == {'b', 'c', 'a'}  # True`

* Implements [ahritmetic of sets](https://en.wikipedia.org/wiki/Arithmetical_set) (union, difference...)
    - `{1, 2, 3, 4} - {1, 3, 5} == {2, 4} == {4, 2}  # True`

* Based on hashing, so the set elements must be hashable (and thus immutable)
* Some mutable datatypes have immutable counterparts...
    - `set` → `frozenset`
    - `list` → `tuple`

* You can iterate over sets, but not access individual members

In [1]:
my_set = {1, 2, 3, 4, 5}
my_set[0]

TypeError: 'set' object is not subscriptable

* Members are guaranteed to be unique within the set (it can't contain duplicate values)

## Using the unorderedness to our advantage

Social network of friends; connections is always bilateral

In [2]:
friendships = [{'Anne', 'Bob'},
               {'Bob', 'Camille'},
               {'Anders', 'Mogens'},
               {'Mogens', 'Donald Duck'},
               {'Darkwind Duck', 'Donald Duck'}]

In [3]:
{'Donald Duck', 'Mogens'} in friendships

True

We feel the need for speed and want $\mathcal{O}(1)$ lookups in the friends variable, rather than the $\mathcal{O}(n)$ as with the list

In [4]:
friendships_set = set(friendships)

TypeError: unhashable type: 'set'

In [5]:
from collections.abc import Hashable
print(isinstance(friendships[0], Hashable)) 

False


### Behold the `frozenset`

In [6]:
# Just like list and dictionary-comprehensions, we have set-comprehensions
friendships_set = {frozenset(relationship) for relationship in friendships}
friendships_set

{frozenset({'Anders', 'Mogens'}),
 frozenset({'Darkwind Duck', 'Donald Duck'}),
 frozenset({'Donald Duck', 'Mogens'}),
 frozenset({'Bob', 'Camille'}),
 frozenset({'Anne', 'Bob'})}

In [7]:
{'Donald Duck', 'Mogens'} in friendships_set

True

## `is in` are fast!

In [8]:
from pathlib import Path
import re
from time import time

def yield_words():
    """Yield books form the Gutenberg project, word by word"""
    p = Path('gutenberg_data')
    rx = re.compile(r'\s+', re.DOTALL)
    for file in p.glob('*.txt'):
        with file.open() as fid:
            yield from rx.split(fid.read().strip())

cnt = 0
for _ in yield_words():
    cnt += 1
    
print(f"We have {cnt:_} words")

We have 3_968_009 words


Get unique words, and preserve the order in which we observed them

In [9]:
%%time

seen_set = set()
seen_list = list()
for word in yield_words():
    if word not in seen_set:
        seen_set.add(word)
        seen_list.append(word)

Wall time: 1.75 s


Get unique words, we don't care about the order

In [10]:
%%time

seen_set = set(yield_words())

Wall time: 1.53 s


We could do this, but the code above is nicer, and a bit faster

```python
seen_set = set()
for word in yield_words():
    seen_set.add(word)
```

Get unique words, using just a list to keep track of observed words.

Cancelled after running it for 1 hour!

In [11]:
%%time

seen_list = list()
for word in yield_words():
    if word not in seen_list:
        seen_list.append(word)

KeyboardInterrupt: 

## Ahrithmetic, and other `set` manipulation


![](./python_sets.png)

In [24]:
s = {1, 2, 3, 4}
t = {1, 3, 5, 5, 5, 5, 5, 5}
print(f"{s=}", f"{t=}", sep='\n') 

s={1, 2, 3, 4}
t={1, 3, 5}


In [25]:
# Set difference...
assert s.difference(t) == s - t
s - t == {2, 4} == {4, 2} 

True

In [26]:
# ... is NOT symmetric
assert s - t != t - s
t - s

{5}

In [27]:
# Set union
assert s.union(t) == s | t
s | t

{1, 2, 3, 4, 5}

In [28]:
# Set symmetric difference (elements unique to either set)
assert s.symmetric_difference(t) == s ^ t
s ^ t

{2, 4, 5}

In [29]:
# Set intersection (elements in both sets)
assert s.intersection(t) == s & t
s & t

{1, 3}

In [30]:
# Set disjoint (check for no members in common)
assert len({'a', 'b'} & {'c', 'd'}) == 0
{'a', 'b'}.isdisjoint({'c', 'd'})

True

In [31]:
# subset, superset and equality
a, b1, b2 = {1, 2}, {1, 2, 3, 4}, {4, 2, 1, 3} 

a.issubset(b1) 
a < b1 

True

In [32]:
b2.issuperset(a)
b2 > a

True

In [33]:
b1 > b2

False

In [34]:
b1 <= b2

True

In [35]:
b1 == b2

True

### Adding and removing from sets

In [36]:
from copy import copy
s1, t1 = copy(s), copy(t)

# Adding members
s1.add('some new element')
s1

{1, 2, 3, 4, 'some new element'}

In [37]:
# Removing members
s1.remove('some new element')
s1

{1, 2, 3, 4}

In [38]:
# Remove member, but no complaining if it's not in the set
s1.discard(42)
s1

{1, 2, 3, 4}

In [39]:
# We can also pop a random member

for _ in range(len(s1)+1):
    print(s1.pop()) 

1
2
3
4


KeyError: 'pop from an empty set'

### Inplace operations

In [40]:
s1, t1 = copy(s), copy(t)
s2, t2 = copy(s), copy(t)

# inplace operation
s1.update(t1) 
s2 |= t2

s2

{1, 2, 3, 4, 5}

In [41]:
s1, t1 = copy(s), copy(t)
s2, t2 = copy(s), copy(t)

s1.intersection_update(t1)  # or (t1, t2, t3, t4, ...)
s1 &= t2 
s1

{1, 3}

In [42]:
s1, t1 = copy(s), copy(t)
s2, t2 = copy(s), copy(t)

s1.difference_update(t1)  # or (t1, t2, t3, t4, ...)
s1 -= t2 
s1

{2, 4}

In [43]:
s1, t1 = copy(s), copy(t)
s2, t2 = copy(s), copy(t)

s1.symmetric_difference_update(t1)
s1 ^= t2
s1

{1, 2, 3, 4}

## How are `set`s related to `dict`s?

* Both are based on [hashtables](https://runestone.academy/runestone/books/published/pythonds/SortSearch/Hashing.html)

* Some of the set-ahritmetic will also work in dict keys in Python 3.9

* Dict are mostly sets with associated values
    - Though dicts mostly retain order, whereas sets are unordered
    - The CPython implementation is different

## The `collections`-module

### Dict-like features that you're likely to need

Let's make a `dict` mapping a person by name to all friends (`Dict[str: Set[str]]`)

In [44]:
import collections

donald_friends = collections.defaultdict(set)

donald_friends['Donald Duck'].add('Mogens')
donald_friends['Donald Duck'].add('Darkwind Duck')

donald_friends

defaultdict(set, {'Donald Duck': {'Darkwind Duck', 'Mogens'}})

Let's count the number of occurences of words in the Gutenberg books

In [45]:
%%time
cnt = collections.Counter(yield_words())
cnt.most_common(15)

Wall time: 1.76 s


[('the', 222850),
 ('of', 130009),
 ('and', 117380),
 ('to', 91597),
 ('a', 79994),
 ('in', 63597),
 ('I', 39431),
 ('was', 36433),
 ('that', 35202),
 ('with', 32052),
 ('is', 28822),
 ('his', 27652),
 ('he', 26003),
 ('as', 24237),
 ('for', 23864)]

## _Questions?_

![](./python_sets.png)